In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore") 

from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.arima.model import ARIMA

import xgboost as xgb

from sklearn.linear_model import LinearRegression

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import select

import os
from dotenv import load_dotenv

In [ ]:
# DEFINE THE DATABASE CREDENTIALS
load_dotenv('env.env')
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

#replace <user>, <password>, <host>, and <port> with your MySQL credentials
cnx = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

## Preparing Data

In [ ]:
# Read Tables in
df = pd.read_sql_table('ElecNetGen', cnx)
df2 = pd.read_sql_table('USA_Monthly_Avg_Temp', cnx)

# Put Temp column into main dataframe
df['Temp'] = df2['Monthly Avg Temp-F']

In [ ]:
# Create Date column with proper formatting from period columns
df['Date'] = pd.to_datetime(df['period'], format="%Y-%m")

# set index to Date
df.set_index(df['Date'], inplace=True)
df.drop('Date', axis=1, inplace=True)
df.drop('period', axis=1, inplace=True)

# Saying treat the index as months ('MS') and start at row 1 instead of the headings .fe1
df.index.fe1= 'MS'

In [ ]:
# Create a variable to manager iterations through the columns
cols = ['Coal', 'Hydroelectric','NaturalGas' , 'Nuclear', 'Petroleum','Temp']

In [ ]:
# Method to perform Augmented Dickey Fuller test - stationary if p-value below less than 0.05
def Augmented_Dickey_Fuller(data):
    res = adfuller(data)
    print(f'p-value: {res[1]}')

In [ ]:
df = df.drop(['Wind', 'Solar'],axis=1)

In [ ]:
# Difference data
diffquant=1
for col in cols:
    df[f'diff_{col}'] = df[col].diff(periods=diffquant).dropna()
    
# Delete NaN rows from differencing data so doesn't error out models
df = df.iloc[diffquant: , :]

## Augmented Dickey-Fuller Test

In [ ]:
# Differenced columns
cols = ['diff_Coal', 'diff_Hydroelectric', 'diff_NaturalGas', 'diff_Nuclear', 'diff_Petroleum', 'diff_Temp']

In [ ]:
# Check data is stationary
for col in cols:
    print(f'{col} ')
    Augmented_Dickey_Fuller(df[col])

In [ ]:
# Create train and test set for both datasets
test_count = 60
train = df.iloc[:-test_count].copy()
test = df.iloc[-test_count:].copy()

# Used to index original dataframe for both the train and test sets
train_index = df.index <= train.index[-1]
test_index = df.index > train.index[-1]

In [ ]:
# Iterate through columns and create a scaled version of them so we can compare
scaler_dic = {}
for col in cols:
    scaler_col = StandardScaler()
    scaler_dic[f"Scaled_{col}"] = scaler_col
    train[f'Scaled_{col}'] = scaler_col.fit_transform(train[[col]])
    test[f'Scaled_{col}'] = scaler_col.transform(test[[col]])

In [ ]:
# Variables for scaled and stationary data
cols = ['Scaled_diff_Coal', 'Scaled_diff_Hydroelectric', 'Scaled_diff_NaturalGas', 'Scaled_diff_Temp',
 'Scaled_diff_Nuclear', 'Scaled_diff_Petroleum']

In [ ]:
# Add scaled features to original df
for col in cols:
    df.loc[train_index, col] = train[col]
    df.loc[test_index, col] = test[col]

In [ ]:
# Add in time components as features
def create_features(df):
    df = df.copy()
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    return df
# Add in time as features
df = create_features(df)
train = create_features(train)
test = create_features(test)

## Clean DataFrame for forecasting

In [ ]:
TARGET = 'Scaled_diff_Temp'

arima = ARIMA(df[TARGET], order=(24,0,24))
arima_result=arima.fit()
prediction_result=arima_result.get_forecast(test_count)

# Create dataframe for 5 year forecast
future = pd.date_range('2023-01-01', '2027-12-01', freq = 'MS')
future_df = pd.DataFrame(index=future)
future_df = create_features(future_df)

# For ARIMA
future_df['pred_Temp'] = predictions = prediction_result.predicted_mean

In [ ]:
# columns for all models to use
cols = ['Scaled_diff_Coal', 'Scaled_diff_Hydroelectric', 'Scaled_diff_NaturalGas', 'Scaled_diff_Temp',
 'Scaled_diff_Nuclear', 'Scaled_diff_Petroleum']
original_cols = ['Coal','Hydroelectric', 'NaturalGas', 'Temp', 'Nuclear', 'Petroleum']

## ARIMA

In [ ]:
ARIMA_df = test.copy()
i = 0

for col in cols:
    TARGET = col
    
    arima = ARIMA(train[col], order=(12,0,12))
    arima_result=arima.fit()
    ARIMA_prediction_result=arima_result.get_forecast(test_count)
    ARIMA_predictions = ARIMA_prediction_result.predicted_mean.to_numpy()
    
    # Scale back data
    ARIMA_df[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(ARIMA_predictions.reshape(-1,1))

    ARIMA_forecast = []
    ARIMA_last_value = train.iloc[-1][f'{original_cols[i]}']
    for x in range(len(test)):
        ARIMA_hold = ARIMA_df[f'pred_inv_{col}'][-test_count:][x] + ARIMA_last_value
        ARIMA_forecast.append(ARIMA_hold)
        ARIMA_last_value = ARIMA_hold
    ARIMA_df[f'forecast_{original_cols[i]}'] = ARIMA_forecast

    ARIMA_disp = [original_cols[i], f'forecast_{original_cols[i]}']
    ARIMA_df[ARIMA_disp].plot(figsize=(10,5), title=f'ARIMA {original_cols[i]}',ylabel='Million Kilowatthours')


    i += 1
    
    # Checks mean squared and r-squared
    print(f"{col} Train RMSE:", mean_squared_error(df.loc[train_index,col],  arima_result.fittedvalues, squared=False))
    print(f"{col} Test RMSE:", round(mean_squared_error(df.loc[test_index,col], ARIMA_prediction_result.predicted_mean,squared=False),4))
    print(f"{col} Train R^2:", r2_score(df.loc[train_index,col],  arima_result.fittedvalues))
    print(f"{col} Test R^2:", round(r2_score(df.loc[test_index,col], ARIMA_prediction_result.predicted_mean),4))

## 5-Year Forecast

In [ ]:
i = 0
ARIMA_future_df = future_df.copy()
for col in cols:
    TARGET = col

    arima = ARIMA(df[TARGET], order=(24,0,24))
                                     
    arima_fcast_res=arima.fit()
    ARIMA_fcast_pred_res=arima_fcast_res.get_forecast(test_count)
    ARIMA_fcast_pred = ARIMA_fcast_pred_res.predicted_mean.to_numpy()

    # Scale back data
    ARIMA_future_df[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(ARIMA_fcast_pred.reshape(-1,1))
    
    # Last - known train value
    ARIMA_fcast = []
    ARIMA_fcast_last_value = test.iloc[-1][original_cols[i]]
    for x in range(len(test)):
        ARIMA_fcast_hold = ARIMA_future_df[f'pred_inv_{col}'][-test_count:][x] + ARIMA_fcast_last_value
        ARIMA_fcast.append(ARIMA_fcast_hold)
        ARIMA_fcast_last_value = ARIMA_fcast_hold
    ARIMA_future_df[f'forecast_{original_cols[i]}'] = ARIMA_fcast

    ARIMA_disp = [f'forecast_{original_cols[i]}']
    ARIMA_future_df[ARIMA_disp].plot(figsize=(15,5))
    i += 1

## VARMAX

In [ ]:
# columns for VARMAX
VARMAX_cols = cols.copy()
VARMAX_original_cols = original_cols.copy()
VARMAX_cols.remove('Scaled_diff_Temp')
VARMAX_original_cols.remove('Temp')

In [ ]:
import pickle 

# Use this to train model and save models in pickle file so don't need to retrain each time.

# for col in VARMAX_cols:
#     points = [col, 'Scaled_diff_Temp']
#     model = VARMAX(train[points],order = (12,12))
#     res = model.fit(maxiter=100)
#     with open(f'{col}_results_24.pkl', 'wb') as file:
#         pickle.dump(res, file)

In [ ]:
import pickle 

VARMAX_df = test.copy()
i = 0

for col in VARMAX_cols:
    TARGET = col
    
    with open(f'{col}_results.pkl', 'rb') as file: 
        VARMAX_res = pickle.load(file)
    VARMAX_fcast = VARMAX_res.get_forecast(test_count)
    
    VARMAX_df[f'{col}_prediction'] = VARMAX_fcast.predicted_mean[col]
    VARMAX_predictions = VARMAX_df[f'{col}_prediction'].to_numpy()
    
    # Scale back data
    VARMAX_df[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(VARMAX_predictions.reshape(-1,1))
    
    VARAMX_forecast = []
    VARMAX_last = train.iloc[-1][f'{VARMAX_original_cols[i]}']
    for x in range(len(test)):
        VARMAX_hold = VARMAX_df[f'pred_inv_{col}'][-test_count:][x] + VARMAX_last
        VARAMX_forecast.append(VARMAX_hold)
        VARMAX_last = VARMAX_hold
    VARMAX_df[f'forecast_{VARMAX_original_cols[i]}'] = VARAMX_forecast

    VARMAX_disp = [VARMAX_original_cols[i], f'forecast_{VARMAX_original_cols[i]}']
    VARMAX_df[VARMAX_disp].plot(figsize = (10,5), title=(f'VARMAX {VARMAX_original_cols[i]}'),
                                    ylabel='Million Kilowatthours')

    i += 1

    # Checks mean squared and r-squared
    print(f"{col} Train RMSE:", mean_squared_error(df.loc[train_index, col],VARMAX_res.fittedvalues[col],squared=False))
    print(f"{col} Test RMSE:", round(mean_squared_error(df.loc[test_index,col], VARMAX_fcast.predicted_mean[col],squared=False),4))
    print(f"{col} Train R^2:", r2_score(df.loc[train_index, col],VARMAX_res.fittedvalues[col]))
    print(f"{col} Test R^2:", round(r2_score(df.loc[test_index,col], VARMAX_fcast.predicted_mean[col]),4))

## VARMAX 5-Year Forecast

In [ ]:
# for col in VARMAX_cols:
#     points = [col, 'Scaled_diff_Temp']
#     VARMAX_fcast_model = VARMAX(df[points],order = (12,12))
#     VARMAX_fcast_res = VARMAX_fcast_model.fit(maxiter=100)
#     with open(f'fcast_{col}_results.pkl', 'wb') as file:
#         pickle.dump(VARMAX_fcast_res, file)

In [ ]:
import pickle 

VARMAX_future_df = future_df.copy()
i = 0

for col in VARMAX_cols:
    TARGET = col
    
    with open(f'fcast_{col}_results.pkl', 'rb') as file: 
        VARMAX_fcast_res = pickle.load(file)
    VARMAX_5_fcast = VARMAX_fcast_res.get_forecast(test_count)
    
    VARMAX_future_df[f'{col}_prediction'] = VARMAX_5_fcast.predicted_mean[col]
    VARMAX_5_predictions = VARMAX_future_df[f'{col}_prediction'].to_numpy()
    
    # Scale back data
    VARMAX_future_df[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(VARMAX_5_predictions.reshape(-1,1))
    
    VARAMX_5_forecast = []
    VARMAX_5_last = test.iloc[-1][f'{VARMAX_original_cols[i]}']
    for x in range(len(test)):
        VARMAX_5_hold = VARMAX_future_df[f'pred_inv_{col}'][-test_count:][x] + VARMAX_5_last
        VARAMX_5_forecast.append(VARMAX_5_hold)
        VARMAX_5_last = VARMAX_5_hold
    VARMAX_future_df[f'forecast_{VARMAX_original_cols[i]}'] = VARAMX_5_forecast

    VARMAX_5_disp = [f'forecast_{VARMAX_original_cols[i]}']
    VARMAX_future_df[VARMAX_5_disp].plot(figsize=(15,5))
    i += 1

## XGBoost

In [ ]:
xgb_RMSE_Scores = []
xgb_df = df.copy()
xgb_test = test.copy()
i = 0

for col in cols:

    TARGET = col
    X_train = train[['quarter', 'month','year', 'Scaled_diff_Temp']]
    y_train = train[TARGET]
    X_test = test[['quarter', 'month','year', 'Scaled_diff_Temp']]
    y_test = test[TARGET]

    # Create the model
    xgb_res = xgb.XGBRegressor(booster='gbtree',    
                           n_estimators=800,
                           early_stopping_rounds=40,
                           objective='reg:squarederror',
                           max_depth=3,
                           learning_rate=0.05)
    xgb_res.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],verbose=False)

    # Forecast on Test and Train
    xgb_test['prediction'] = xgb_res.predict(X_test)
    xgb_df.loc[train_index,'train_prediction'] = xgb_res.predict(X_train)
    xgb_pred = xgb_test['prediction'].to_numpy()
    xgb_train_pred = xgb_df.loc[train_index,'train_prediction'].to_numpy()

    # Scale back data
    xgb_test[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(xgb_pred.reshape(-1,1))

    # Last - known train value
    xgb_forecast = []
    xgb_last = train.iloc[-1][f'{original_cols[i]}']
    for x in range(len(test)):
        xgb_hold = xgb_test[f'pred_inv_{col}'][-test_count:][x] + xgb_last
        xgb_forecast.append(xgb_hold)
        xgb_last = xgb_hold
    xgb_test[f'forecast_{original_cols[i]}'] = xgb_forecast

    xgb_disp = [original_cols[i], f'forecast_{original_cols[i]}']
    xgb_test[xgb_disp].plot(figsize=(10,5), title= f'XGB {original_cols[i]}',ylabel='Million Kilowatthours')


    i += 1

    # Checks mean squared and r-squared
    print(f"{col} Train RMSE:", mean_squared_error(train[TARGET], xgb_df.loc[train_index,'train_prediction'], squared=False))
    print(f"{col} Test RMSE:", round(mean_squared_error(test[TARGET], xgb_test['prediction'],squared=False),4))
    print(f"{col} Train R^2:", r2_score(train[TARGET],  xgb_df.loc[train_index,'train_prediction']))
    print(f"{col} Test R^2:", round(r2_score(test[TARGET], xgb_test['prediction']),4))

### 5 Year Forecast

In [ ]:
future_df_xgb = future_df.copy()
future_df_xgb.rename({'pred_Temp':'Scaled_diff_Temp'},axis=1, inplace=True)
i = 0

for col in cols:
    FEATURES = ['quarter', 'month', 'year', 'Scaled_diff_Temp']
    TARGET = col

    X_all = df[FEATURES]
    y_all = df[TARGET]

    # Create the model
    xgb_fcast_reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=800,
                           early_stopping_rounds=40,
                           objective='reg:squarederror',
                           max_depth=3,
                           learning_rate=0.05)
    xgb_fcast_reg.fit(X_all, y_all, eval_set=[(X_all, y_all)], verbose=False)

    # For XGBoost
    future_df_xgb['pred'] = xgb_fcast_pred = xgb_fcast_reg.predict(future_df_xgb[FEATURES])

    # Scale back data
    future_df_xgb[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(xgb_fcast_pred.reshape(-1,1))
    
    # Last - known train value
    xgb_fcast = []
    xgb_fcast_last_value = test.iloc[-1][original_cols[i]]
    for x in range(len(test)):
        xgb_fcast_hold = future_df_xgb[f'pred_inv_Scaled_diff_{original_cols[i]}'][-test_count:][x] + xgb_fcast_last_value
        xgb_fcast.append(xgb_fcast_hold)
        xgb_fcast_last_value = xgb_fcast_hold
    future_df_xgb[f'forecast_{original_cols[i]}'] = xgb_fcast

    xgb_fcast_disp = [f'forecast_{original_cols[i]}']
    future_df_xgb[xgb_fcast_disp].plot(figsize=(15,5))
    i += 1

## Linear Regression

In [ ]:
lr_RMSE_Scores = []
i = 0
lr_test = test.copy()

for col in cols:
    TARGET = col
    lr_model = LinearRegression()
    X_train = train[['Scaled_diff_Temp', 'quarter', 'month', 'year']]
    y_train = train[TARGET]
    X_test = test[['Scaled_diff_Temp', 'quarter', 'month', 'year']]
    y_test = test[TARGET]
    
    # fitting the model 
    lr_model.fit(X_train,y_train)

    # making predictions 
    lr_pred_train = lr_model.predict(X_train)
    lr_test['prediction'] = lr_pred = lr_model.predict(X_test)
    
    # Scale back data
    lr_test[f'pred_inv_{col}'] = scaler_dic[TARGET].inverse_transform(lr_pred.reshape(-1,1))

    # Last - known train value
    lr_forecast = []
    lr_last_value = train.iloc[-1][f'{original_cols[i]}']
    for x in range(len(test)):
        lr_hold = lr_test[f'pred_inv_{col}'][-test_count:][x] + lr_last_value
        lr_forecast.append(lr_hold)
        lr_last_value = lr_hold
    lr_test[f'forecast_{original_cols[i]}'] = lr_forecast

    lr_disp = [original_cols[i], f'forecast_{original_cols[i]}']
    lr_test[lr_disp].plot(figsize=(10,5), title=f'LR_{original_cols[i]}',ylabel='Million Kilowatthours')

    
    i += 1
    
    # Checks mean squared and r-squared
    print(f"{col} Train RMSE:", mean_squared_error(y_train, lr_pred_train, squared=False))
    print(f"{col} Test RMSE:", round(mean_squared_error(y_test, lr_pred, squared=False),4))
    print(f"{col} Train R^2:", r2_score(y_train, lr_pred_train))
    print(f"{col} Test R^2:", round(r2_score(y_test, lr_pred),4))

### 5-Year Forecast

In [ ]:
# Create dataframe for 5 year forecast
future_df_lr = future_df.copy()
future_df_lr.rename({'pred_Temp':'Scaled_diff_Temp'},axis=1, inplace=True)
i = 0

for col in cols:
    FEATURES = ['quarter', 'month', 'year', 'Scaled_diff_Temp']
    TARGET = col

    X_all = df[FEATURES]
    y_all = df[TARGET]

    # Create the model
    lr_fcast_model = LinearRegression()
    
    lr_fcast_model.fit(X_all, y_all)

    # For LR
    future_df_lr['pred'] = lr_fcast_pred = lr_fcast_model.predict(future_df_lr[FEATURES])

    # Scale back data
    future_df_lr[f'pred_inv_{cols}'] = scaler_dic[TARGET].inverse_transform(lr_fcast_pred.reshape(-1,1))
    
    # Last - known train value
    lr_5_fcast = []
    lr_5_last_value = test.iloc[-1][original_cols[i]]
    for x in range(len(test)):
        lr_5_hold = future_df_lr[f'pred_inv_{cols}'][-test_count:][x] + lr_5_last_value
        lr_5_fcast.append(lr_5_hold)
        lr_5_last_value = lr_5_hold
    future_df_lr[f'forecast_{original_cols[i]}'] = lr_5_fcast

    lr_5_disp = [f'forecast_{original_cols[i]}']
    future_df_lr[lr_5_disp].plot(figsize=(15,5))
    i += 1